In [1]:
import sys
import os
os.chdir("../../")

print(sys.version)

3.8.2 (default, Mar 25 2020, 17:03:02) 
[GCC 7.3.0]


In [2]:
import slim

In [3]:
"""
A script that downloads and parses Salmon Scotland mortality reports.
"""

from bs4 import BeautifulSoup
from requests import get
import os
from pathlib import Path
import tabula
import numpy as np

WEBSITE = "https://www.salmonscotland.co.uk"
REPORT_URL = f"{WEBSITE}/reports/monthly-mortality-rate-%s-%d"


def download(month: str, year: int):
    report_out_folder = Path("output/reports/")
    filename = report_out_folder / f"SS-{month}-{year}.pdf"
    os.makedirs(str(report_out_folder), exist_ok=True)
    
    if not filename.exists():
        url = REPORT_URL % (month, year)
        parse_page = get(url).content
        parser = BeautifulSoup(parse_page, "html.parser")
        div = parser.find("div", class_="download-link")
        a = div.find("a")
        download_link = WEBSITE + a["href"]

        report_out_folder = Path("output/reports/")
        filename = report_out_folder / f"SS-{month}-{year}.pdf"
        os.makedirs(str(report_out_folder), exist_ok=True)
        downloaded_pdf = get(download_link)

        with filename.open("wb") as f:
            f.write(downloaded_pdf.content)

    return filename

In [219]:
import pandas as pd

column_names = ["company", "site", "mortality", "note", "cumulative_mortality"]

def get_pdf_page(pdf, page, month, year):
    # parsing of multiple pages breaks
    candidate = tabula.read_pdf(pdf, pages=page, pandas_options={"header": "none"})
    
    if len(candidate) == 0:
        return pd.DataFrame({column: {} for column in column_names})
    df = candidate[0]
    if page == 2:
        df = df.iloc[3:].reset_index(drop=True)
    
    if len(df.columns) == 4:
        cumulative_mort = df[3].copy()
        df[3] = np.nan
        df[4] = cumulative_mort
    elif len(df.columns) == 3:
        df[3] = np.nan
        df[4] = np.nan
        
    df = df.set_axis(column_names, axis=1)
    

    df["month"] = month
    df["year"] = year
    df["year"] = df["year"].astype(np.int64)
    return df

def get_pdf_pages(pdf, month, year):
    if (month == "December" and year == 2020):
        range_ = range(0, 6)
    else:
        range_ = range(2, 8)
    dfs = [get_pdf_page(pdf, page, month, year) for page in range_]
    return pd.concat(dfs, ignore_index=True, sort=False)

In [220]:
import asyncio
import datetime

def extract_year_report(year):
    def parse_and_extract(month):
        month_label = datetime.date(year, month, 1).strftime('%B')
        location = download(month_label, year)
        return get_pdf_pages(location, month_label, year)
    
    dfs = [parse_and_extract(month) for month in range(1, 13)]
    df = pd.concat(dfs, ignore_index=True)
    df[~df["company"].isnull()].reset_index(drop=True)
    df["year"] = df["year"].apply(np.uint64)
    
    return df

def collate_years(range_):
    return pd.concat((extract_year_report(y) for y in range_), ignore_index=True, sort=False)

In [221]:
# This is going to take some time

df = collate_years(range(2019, 2021))

'pages' argument isn't specified.Will extract only from page 1 by default.


In [222]:
df

,company,site,mortality,note,cumulative_mortality,month,year
0,Cooke Aquaculture (Scotland),Balta Isle,1.9,NaN,In production,January,2019
1,Cooke Aquaculture (Scotland),Bastaness,0.5,NaN,In production,January,2019
2,Cooke Aquaculture (Scotland),Bay of Cleat (North),Fallow,NaN,Fallow,January,2019
3,Cooke Aquaculture (Scotland),Bay of Ham,Fallow,NaN,Fallow,January,2019
4,Cooke Aquaculture (Scotland),Bay of Vady,3.0,NaN,In production,January,2019
...,...,...,...,...,...,...,...
4635,Scottish Sea Farms Ltd,Wyre,0.01,NaN,NaN,December,2020
4636,Wester Ross Fisheries Ltd,Ardessie A,0.00,NaN,NaN,December,2020
4637,Wester Ross Fisheries Ltd,Ardessie B,0.00,NaN,NaN,December,2020
4638,Wester Ross Fisheries Ltd,Ardmair,0.04,NaN,NaN,December,2020


## Marine Scotland parsing

In [106]:
import json
import re

MS_REPORT_JSON = "https://data.marine.gov.scot/api/3/action/package_show?id=55aa8a12-135e-463e-802b-fb661fa02b73&page=0"
REPORT_JSON_LOCATION = Path("output/reports")

def get_lice_counts_json(year):
    report_json_location = REPORT_JSON_LOCATION / f"MS_{year}.csv"
    if not report_json_location.exists():
        parsed = json.loads(get(MS_REPORT_JSON).content)
        titles_urls = [(res["name"], res["url"]) for res in parsed["result"][0]["resources"]]
        titles, urls = zip(*titles_urls)

        ranges = [range(*tuple(map(int, 
                      re.findall(r"(\d+)-(\d+)", s)[0])
                             )) for s in titles]
        url = next(url for idx, url in enumerate(urls) if year in ranges[idx])
        with report_json_location.open("wb") as f:
            response = get(url, headers={'Content-type': 'application/json'})
            f.write(response.content)

    return report_json_location

def get_lice_counts(year):
    return pd.read_csv(str(get_lice_counts_json(year)))

In [107]:
lice_counts_2019 = get_lice_counts(2019)

https://data.marine.gov.scot/sites/default/files//Sea%20lice%20count%20data%20-%202018-2020_0.csv
<Response [200]>


In [108]:
lice_counts_2019

,Site ID,Site Name,Year,Month,Lice Count,Comments
0,BALT1,Balta Island,2018,January,0.03,NaN
1,BALT1,Balta Island,2018,February,0.07,NaN
2,BALT1,Balta Island,2018,March,0.15,NaN
3,BALT1,Balta Island,2018,April,0.1,NaN
4,BALT1,Balta Island,2018,May,0.06,NaN
...,...,...,...,...,...,...
8274,BRO1,"Corry, Loch Broom",2020,December,0,NaN
8275,WHA2,North Voe,2020,December,F,NaN
8276,MCLN1,MacLeans Nose,2020,December,0,NaN
8277,KIL1,Petersport South (Kilerivagh),2020,December,F,NaN


In [113]:
lice_counts_2019["Site Name"]

0                        Balta Island
1                        Balta Island
2                        Balta Island
3                        Balta Island
4                        Balta Island
                    ...              
8274                Corry, Loch Broom
8275                        North Voe
8276                    MacLeans Nose
8277    Petersport South (Kilerivagh)
8278                     Puldrite Bay
Name: Site Name, Length: 8279, dtype: object

In [129]:
sites = lice_counts_2019["Site Name"]
    
linnhe_sites_data = lice_counts_2019[sites.str.contains("Linnhe").fillna(False)]

In [224]:
df_linnhe = df[df["site"] == "Linnhe"]

In [225]:
# We are ready to join the two dataframes

linnhe_data_joined = linnhe_sites_data.set_index(["Year", "Month"])\
                    .join(df_linnhe.set_index(["year", "month"]).rename_axis(["Year", "Month"]),
                          how="right")
linnhe_data_joined

Site ID         Site Name Lice Count               Comments  \
Year Month                                                                   
2019 February    ARDG1  Ardgour (Linnhe)       0.41                    NaN   
     March       ARDG1  Ardgour (Linnhe)       0.37                    NaN   
     April       ARDG1  Ardgour (Linnhe)       0.36                    NaN   
     May         ARDG1  Ardgour (Linnhe)       0.37                    NaN   
     June        ARDG1  Ardgour (Linnhe)       0.41                    NaN   
     July        ARDG1  Ardgour (Linnhe)       0.97                    NaN   
     August      ARDG1  Ardgour (Linnhe)        FNC  Farm fallowed in Aug.   
     September   ARDG1  Ardgour (Linnhe)          F                 Fallow   
     October     ARDG1  Ardgour (Linnhe)          F                 Fallow   
     November    ARDG1  Ardgour (Linnhe)        SNC   Farm stocked in Nov.   
     December    ARDG1  Ardgour (Linnhe)          0                    NaN   
2020 January     ARDG1  Ardgour (Linnhe)          0                    NaN   
     February    ARDG1  Ardgour (Linnhe)       0.01                    NaN   
     March       ARDG1  Ardgour (Linnhe)          0                    NaN   
     April       ARDG1  Ardgour (Linnhe)          0                    NaN   
     May         ARDG1  Ardgour (Linnhe)       0.01                    NaN   
     June        ARDG1  Ardgour (Linnhe)       0.02                    NaN   
     July        ARDG1  Ardgour (Linnhe)       0.07                    NaN   
     August      ARDG1  Ardgour (Linnhe)       0.16                    NaN   
     September   ARDG1  Ardgour (Linnhe)       0.04                    NaN   
     October     ARDG1  Ardgour (Linnhe)       0.17                    NaN   
     November    ARDG1  Ardgour (Linnhe)       0.44                    NaN   
     December    ARDG1  Ardgour (Linnhe)       0.81                    NaN   

                              company    site                    mortality  \
Year Month                                                                   
2019 February   Mowi Scotland Limited  Linnhe                          0.7   
     March      Mowi Scotland Limited  Linnhe                          0.5   
     April      Mowi Scotland Limited  Linnhe                          0.5   
     May        Mowi Scotland Limited  Linnhe                          0.6   
     June       Mowi Scotland Limited  Linnhe                          1.3   
     July       Mowi Scotland Limited  Linnhe                         16.2   
     August     Mowi Scotland Limited  Linnhe  1.9 (Farm fallowed in Aug.)   
     September  Mowi Scotland Limited  Linnhe                       Fallow   
     October    Mowi Scotland Limited  Linnhe                       Fallow   
     November   Mowi Scotland Limited  Linnhe                          0.2   
     December   Mowi Scotland Limited  Linnhe                          0.8   
2020 January    Mowi Scotland Limited  Linnhe                          0.4   
     February   Mowi Scotland Limited  Linnhe                          0.1   
     March      Mowi Scotland Limited  Linnhe                          0.2   
     April      Mowi Scotland Limited  Linnhe                          0.4   
     May        Mowi Scotland Limited  Linnhe                          0.2   
     June       Mowi Scotland Limited  Linnhe                          0.1   
     July       Mowi Scotland Limited  Linnhe                          0.2   
     August     Mowi Scotland Limited  Linnhe                          0.3   
     September  Mowi Scotland Limited  Linnhe                          0.2   
     October    Mowi Scotland Limited  Linnhe                          0.2   
     November   Mowi Scotland Limited  Linnhe                          0.1   
     December   Mowi Scotland Limited  Linnhe                         0.81   

               note cumulative_mortality  
Year Month                                
2019 February   NaN        

In [253]:
# final clean-ups

def replace(mortality):
    if (x := re.match("(\d+(\.\d+)?)", mortality)):
        return float(x.group())
    return np.nan

linnhe_df = linnhe_data_joined.copy()
    
linnhe_df["mortality"] = linnhe_data_joined["mortality"].apply(replace)
linnhe_df["lice_count"] = linnhe_data_joined["Lice Count"].apply(replace)
linnhe_df.rename({"Comments": "mortality_comment", "Site ID": "site_id", "note": "lice_note"}, axis=1)\
    [["site_id", "site", "lice_count", "mortality", "mortality_comment", "lice_note"]]

site_id    site  lice_count  mortality      mortality_comment  \
Year Month                                                                     
2019 February    ARDG1  Linnhe        0.41       0.70                    NaN   
     March       ARDG1  Linnhe        0.37       0.50                    NaN   
     April       ARDG1  Linnhe        0.36       0.50                    NaN   
     May         ARDG1  Linnhe        0.37       0.60                    NaN   
     June        ARDG1  Linnhe        0.41       1.30                    NaN   
     July        ARDG1  Linnhe        0.97      16.20                    NaN   
     August      ARDG1  Linnhe         NaN       1.90  Farm fallowed in Aug.   
     September   ARDG1  Linnhe         NaN        NaN                 Fallow   
     October     ARDG1  Linnhe         NaN        NaN                 Fallow   
     November    ARDG1  Linnhe         NaN       0.20   Farm stocked in Nov.   
     December    ARDG1  Linnhe        0.00       0.80                    NaN   
2020 January     ARDG1  Linnhe        0.00       0.40                    NaN   
     February    ARDG1  Linnhe        0.01       0.10                    NaN   
     March       ARDG1  Linnhe        0.00       0.20                    NaN   
     April       ARDG1  Linnhe        0.00       0.40                    NaN   
     May         ARDG1  Linnhe        0.01       0.20                    NaN   
     June        ARDG1  Linnhe        0.02       0.10                    NaN   
     July        ARDG1  Linnhe        0.07       0.20                    NaN   
     August      ARDG1  Linnhe        0.16       0.30                    NaN   
     September   ARDG1  Linnhe        0.04       0.20                    NaN   
     October     ARDG1  Linnhe        0.17       0.20                    NaN   
     November    ARDG1  Linnhe        0.44       0.10                    NaN   
     December    ARDG1  Linnhe        0.81       0.81                    NaN   

               lice_note  
Year Month                
2019 February        NaN  
     March           NaN  
     April           NaN  
     May             NaN  
     June            NaN  
     July            CMS  
     August          NaN  
     September       NaN  
     October         NaN  
     November        NaN  
     December        NaN  
2020 January         NaN  
     February        NaN  
     March           NaN  
     April           NaN  
     May             NaN  
     June            NaN  
     July            NaN  
     August          NaN  
     September       NaN  
     October         NaN  
     November        NaN  
     December        NaN

In [267]:
linnhe_df.to_csv("config_data/Linnhe_complete/report.csv")